In [1]:
################################################################################################
####
####       Solids classificator
####       @ NeuroneLab - University of Salerno  - IT
####       dott. Francesco Bardozzo    : fbardozzo@unisa.it
####       dott. Gioele Ciaparrone     : gciaparrone@unisa.it
###        dott. Mattia Delli Priscoli : mdellipriscoli@unisa.it
#################################################################################################

#!/usr/bin/env python
# coding: utf-8

# In[1]:

from __future__ import print_function
import os

import numpy as np
import tensorflow as tf
import math
from numpy.random import RandomState
from random import shuffle
from progressbar import *
import time
import GPUtil

np.random.seed(17)

# In[2]:
#InteractiveSession.close()

# Set the Graph
g = tf.Graph()

# Define the CNN parameters
epochs = 10

batch_val = 10
n_batches = 70
#		"data_dir":'/home/francis/Scrivania/NeuroImagingProject2018/PythonSource',
width = 32
height = 32
depth = 32
exp_size = 700
test_size = 300

with g.as_default():
	batch_size = tf.placeholder(tf.int64, name="batch")
	keep_prob = tf.placeholder(tf.float32, name="keepprob")

# In[3]:


# Tensorflow working version: 1.5.1
print("Tensorflow working version: %s" % tf.__version__)

import sys
print("Python version:", sys.version)


        


Tensorflow working version: 1.8.0
Python version: 3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]


In [2]:
# Set CUDA_DEVICE_ORDER so the IDs assigned by CUDA match those from nvidia-smi
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

# Get the first available GPU
DEVICE_ID_LIST = GPUtil.getFirstAvailable()
# grab first element from list
DEVICE_ID = DEVICE_ID_LIST[0]

print(DEVICE_ID)
# Set CUDA_VISIBLE_DEVICES to mask out all other GPUs than the first available device id
os.environ["CUDA_VISIBLE_DEVICES"] = str(DEVICE_ID)

# Since all other GPUs are masked out, the first available GPU will now be identified as GPU:0
print('Device ID (unmasked): ' + str(DEVICE_ID))
print('Device ID (masked): ' + str(0))
with tf.device("/GPU:0"):
	sess = tf.InteractiveSession(graph=g)
	print("Interactive session on GPU initialized.")

# In[4]:


print("Number of Epochs: %d" % (epochs))
print("Batch size: %d" % (batch_val))
print('Number of batches for each epoch: %d' % (exp_size / batch_val))

# In[5]:

0
Device ID (unmasked): 0
Device ID (masked): 0
Interactive session on GPU initialized.
Number of Epochs: 10
Batch size: 10
Number of batches for each epoch: 70


In [3]:
with g.as_default():
	x, y = tf.placeholder(tf.float32, shape=[None, width, height, depth], name='input'), tf.placeholder(tf.float32,
																										shape=[None, 1],
																										name='labels')
	train_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).repeat()
	test_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size)

	print("x: Shape of the train  set: %s" % (x.shape))
	print("y: Shape of the labels set: %s" % (y.shape))

	iter = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
	# Definition of the iterator, features are ginven in input to the fist convolutional layer.
	features, labels = iter.get_next()
	train_init_op = iter.make_initializer(train_dataset)
	test_init_op = iter.make_initializer(test_dataset)

# In[6]:

x: Shape of the train  set: (?, 32, 32, 32)
y: Shape of the labels set: (?, 1)


In [4]:
# Training set random generation:
train_set_data = []
# rr = RandomState()
#Load the spheres
for i in range(int(exp_size / 2)):
	# FA_data = rr.uniform(0, 300, size=(width, height, depth))
	# print(FA_data.shape)
	FA_data = np.load("./data_solid/%d_s.npy" % i, encoding = 'bytes')
	train_set_data = np.append(train_set_data, np.asarray(FA_data, dtype='float32'))

for i in range(int(exp_size / 2), exp_size):
	# FA_data = rr.uniform(0, 300, size=(width, height, depth))
	# print(FA_data.shape)
	FA_data = np.load("./data_solid/%d_o.npy" % i, encoding = 'bytes')
	train_set_data = np.append(train_set_data, np.asarray(FA_data, dtype='float32'))

# reshape training set
train_set_data = train_set_data.reshape(exp_size, width, height, depth)
print("Random generation of training set, with shape:", (train_set_data.shape))
# label_train = np.random.randint(2, size=(exp_size, 1))
# print("Random generation of labels for trainign set, with shape", label_train.shape)
label_train = np.load("./labels_solid/label_train_set.npy", encoding = 'bytes').reshape((exp_size,1))
print("Random generation of labels for trainign set, with shape", label_train.shape)
# Training set


indices = np.arange(exp_size)
np.random.shuffle(indices)


train_set_data = train_set_data[indices]
label_train = label_train[indices]

train_data = (train_set_data, label_train)

# In[7]:


# Test set random generation:

test_set_data = []
# rr = RandomState()
for i in range(int(test_size / 2)):
	# FLAGS.exp_size
	# FA_data = rr.uniform(0, 300, size=(width, height, depth))
	FA_data = np.load("./test_solid/%d_s.npy" % i, encoding = 'bytes')
	test_set_data = np.append(test_set_data, np.asarray(FA_data, dtype='float32'))

for i in range(int(test_size / 2), test_size):
	# FLAGS.exp_size
	# FA_data = rr.uniform(0, 300, size=(width, height, depth))
	FA_data = np.load("./test_solid/%d_o.npy" % i, encoding = 'bytes')
	test_set_data = np.append(test_set_data, np.asarray(FA_data, dtype='float32'))
# reshape test set
test_set_data = test_set_data.reshape(test_size, width, height, depth)
print("Random generation of test set, with shape:", (test_set_data.shape))
# label_test = np.random.randint(2, size=(test_size, 1))
# print("Random generation of labels for trainign set, with shape", label_test.shape)
label_test = np.load("./labels_solid/label_test_set.npy", encoding = 'bytes').reshape((test_size,1))
print("Random generation of labels for trainign set, with shape", label_test.shape)
# Test set
test_data = (test_set_data, label_test)




Random generation of training set, with shape: (700, 32, 32, 32)
Random generation of labels for trainign set, with shape (700, 1)
Random generation of test set, with shape: (300, 32, 32, 32)
Random generation of labels for trainign set, with shape (300, 1)


In [5]:
test_set_data.dump("test_set_data.npy")
label_test.dump("label_test.npy")

train_set_data.dump("train_set_data.npy")
label_train.dump("label_train.npy")

In [6]:
# Model function definition:
def weight_variable(shape, name):
	initial = tf.truncated_normal(shape, stddev=0.1, name=name)
	return tf.Variable(initial)


def bias_variable(shape, name):
	initial = tf.constant(0.1, shape=shape, name=name)
	return tf.Variable(initial)


# Convolution 3D-Tensor
def conv3d(x, W, name):
	# stride [1, x_movement, y_movement, z_,movement 1]
	return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1, 1], padding='SAME', name=name)


# Pooling: max pooling over 2x2 blocks
def max_pool_2x2(x, name):
	# stride [1, x_movement, y_movement, z_movement, 1]
	return tf.nn.max_pool3d(x, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding='SAME', name=name)


# Input shape
x_image = tf.reshape(features, [-1, width, height, depth, 1], name="input_x")
print("Shape of the input layer: %s" % (x_image.shape))  # [n_samples, 28,28,28,1]

Shape of the input layer: (?, 32, 32, 32, 1)


In [7]:
def plot_slices(tensor, name, max_outputs):
	x_min = tf.reduce_min(tensor)
	x_max = tf.reduce_max(tensor)
	weights_0_to_1 = (tensor - x_min) / (x_max - x_min)
	weights_0_to_255_uint8 = tf.image.convert_image_dtype(weights_0_to_1, dtype=tf.uint8)
	# to tf.image_summary format [batch_size, height, width, channels]
	weights_transposed = tf.transpose(weights_0_to_255_uint8, [4, 0, 1, 2, 3])
	for i in range(32):
		# this will display random 3 filters from the 64 in conv1
		tf.summary.image(name % i, weights_transposed[i,:,:,:,0:1], max_outputs=max_outputs)

## First layer ## downsampling
#with g.as_default():
    #map_feat = 32
    #patch1 = [5, 5, 5, 1, map_feat]

    #W_conv1 = weight_variable(patch1, "W_conv1")  # patch 5x5x5, in size 1, out size 32
    #plot_slices(W_conv1, 'conv1/filters_%d', max_outputs=5)
    #b_conv1 = bias_variable([map_feat], "b_conv1")
    #h_conv1 = tf.nn.relu(conv3d(x_image, W_conv1, name="h_conv1_3d") + b_conv1, name="h_conv1")
    # output size 28x28x28x32
    #plot_slices(h_conv1, 'conv1/out_%d', max_outputs=32)
    #h_pool1 = max_pool_2x2(h_conv1, "h_pool1")  # output size 14x14x14x32
    #print(h_pool1.get_shape)

# In[11]:

In [8]:
# First Layer: Convolutional layer + MaxPooling
## conv1 layer ## downsampling
with g.as_default():
    map_feat = 32
    patch2 = [5, 5, 5, 1, map_feat]
    W_conv2 = weight_variable(patch2, "W_conv2")  # patch 5x5x5, in size 1, out size 32
    plot_slices(W_conv2, 'conv2/filters_%d', max_outputs=5)
    b_conv2 = bias_variable([map_feat], "b_conv2")
    h_conv2 = tf.nn.relu(conv3d(x_image, W_conv2, name="h_conv2_3d") + b_conv2,
                         name="h_conv2")  # output size 28x28x28x32
    h_pool2 = max_pool_2x2(h_conv2, "h_pool2")  # output size 14x14x14x32
    print(h_pool2.get_shape)

# In[12]:


# Third Layer: Convolutional Layer + MaxPooling
with g.as_default():
    map_feat2 = 64
    patch2 = [5, 5, 5, map_feat, map_feat2]

    W_conv3 = weight_variable(patch2, "W_conv3")  # patch 5x5, in size 32, out size 64
    b_conv3 = bias_variable([map_feat2], "b_conv3")
    h_conv3 = tf.nn.relu(conv3d(h_pool2, W_conv3, name="h_conv3_3d") + b_conv3, name="h_conv3")  # output size 14x14x64
    h_pool3 = max_pool_2x2(h_conv3, "h_pool3")  # output size 7x7x64
    print(h_pool3.get_shape)

    h_pool3_s = h_pool3.get_shape().as_list()

# In[13]:


# Fourth Layer: Flattened Layer + Dropout
with g.as_default():
    map_feat3 = 1024
    flat_map = h_pool3_s[1] * h_pool3_s[2] * h_pool3_s[3] * h_pool3_s[4]
    patch3 = [flat_map, map_feat3]
    W_fc1 = weight_variable(patch3, "W_fc1")
    b_fc1 = bias_variable([1024], "b_fc1")
    # [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
    h_pool3_flat = tf.reshape(h_pool3, [-1, flat_map], name="h_pool3_flat")
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1, name="h_pool3_flat_mm") + b_fc1, name="h_fc1")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob, name="h_fc1_drop")

    print(h_fc1_drop.get_shape)

# prediction = tf.layers.dense(h_fc1_drop, 1, activation=tf.tanh)


# In[14]:


# Output layer:
with g.as_default():
    h1 = tf.layers.dense(h_fc1_drop, 1, activation=None, name="output_layer")
    print(h1.get_shape)

# In[15]:


with g.as_default():
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=h1, name="sig_log")
    loss = tf.reduce_mean(tf.cast(cross_entropy, tf.float32), name="red_m")
    learning_rate = 0.01
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    #grads = optimizer.compute_gradients(loss)
    #train_op = optimizer.apply_gradients(grads)
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    prediction = tf.nn.sigmoid(h1)
    correct_prediction = tf.equal(tf.round(prediction), labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    grads = tf.gradients(h1, h_conv3)
    print(prediction.get_shape)

# In[16]:

<bound method Tensor.get_shape of <tf.Tensor 'h_pool2:0' shape=(?, 16, 16, 16, 32) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'h_pool3:0' shape=(?, 8, 8, 8, 64) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'h_fc1_drop/mul:0' shape=(?, 1024) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'output_layer/BiasAdd:0' shape=(?, 1) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Sigmoid:0' shape=(?, 1) dtype=float32>>


In [9]:
with g.as_default():
    adj_loss = 1
    limit = 0.8
    summary = tf.summary.merge_all()
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        summary_writer = tf.summary.FileWriter('tensorboard', sess.graph)
        sess.run(tf.global_variables_initializer())
        # initialise iterator with train data
        print('Training...')
        for i in range(epochs):
            sess.run(train_init_op, feed_dict={x: train_data[0], y: train_data[1], batch_size: batch_val})
            # compute loss & accuracy
            widgets = [FormatLabel(''), ' ', Percentage(), ' ', Bar('/'), ' ', RotatingMarker(), ' ', ETA()]
            progressbar = ProgressBar(widgets=widgets, maxval=n_batches)
            progressbar.start()
            tot_loss = 0
            train_acc = 0
            for k in range(n_batches):

                train_accuracy, _, loss_value , cp, summary_str= sess.run([accuracy, train_op, loss, correct_prediction, summary], feed_dict={keep_prob: 0.5})

                #tensorboard        
                summary_writer.add_summary(summary_str, i*n_batches + k)
                summary_writer.flush()
                #train acc
                train_acc += train_accuracy
                tot_loss += loss_value
                widgets[0] = FormatLabel('Epoch: {0}, Acc: {1:.2f} , Loss: {2:.2f}'.format(i, train_accuracy, tot_loss / adj_loss))
                progressbar.update(k)
            progressbar.finish()
            
            h_out2, h_out3, g_out = sess.run([h_conv2, h_conv3, grads], feed_dict={keep_prob: 1})
            for f,v in zip(["conv2.npy", "conv3.npy", "grads.npy"],[h_out2, h_out3, g_out]):
                with open(f, "wb") as outfile:
                    np.save(outfile, v)

            sess.run(test_init_op, feed_dict={x: test_data[0], y: test_data[1], batch_size: batch_val})
            test_acc = 0
            for k in range(math.ceil(test_size/batch_val)):
                test_accuracy, f, c = sess.run([accuracy, features, correct_prediction], feed_dict={keep_prob: 1})
                #dump missclassifications in last epochs
                for idx in range(c.shape[0]):
                    fel = f[idx]
                    cel = c[idx]
                    if not cel and i > 18:
                        print("Dumping")
                        fel.dump("wrong_pred_%d.npy" % (np.random.randint(10000)))    
                test_acc += test_accuracy
            print("Avg Train Accuracy : {:.4f}".format(train_acc / n_batches))
            print("Test Set Accuracy %.4f " % (test_acc / (test_size/batch_val)))
            print("Utilization GPU")
            GPUtil.showUtilization()
        #if early_stop >= limit:
        #break


        print('Epoch', i, 'Completed out of', epochs, 'loss:', tot_loss / adj_loss)
        # initialise iterator with test data
        sess.run(test_init_op,
                 feed_dict={x: test_data[0], y: test_data[1], batch_size: 20, keep_prob: 1})
        print('Test Loss: {:4f}'.format(sess.run(loss, feed_dict={keep_prob: 1})))

    

 N/A% |                                                      | / ETA:  --:--:--

Training...


Epoch: 0, Acc: 0.90 , Loss: 1792.51 100% |///////////////////| | Time:  0:00:03
Epoch: 1, Acc: 1.00 , Loss: 0.73   1% |                      | - ETA:   0:00:05

Avg Train Accuracy : 0.6757
Test Set Accuracy 0.9133 
Utilization GPU
| ID | GPU | MEM |
------------------
|  0 | 91% | 24% |


Epoch: 1, Acc: 1.00 , Loss: 8.08 100% |//////////////////////| | Time:  0:00:02
Epoch: 2, Acc: 1.00 , Loss: 0.02   1% |                      | - ETA:   0:00:05

...

Avg Train Accuracy : 1.0000
Test Set Accuracy 0.9967 
Utilization GPU
| ID | GPU | MEM |
------------------
|  0 | 91% | 24% |


Epoch: 8, Acc: 1.00 , Loss: 0.01 100% |//////////////////////| | Time:  0:00:02
Epoch: 9, Acc: 1.00 , Loss: 0.00   1% |                      | - ETA:   0:00:05

Avg Train Accuracy : 1.0000
Test Set Accuracy 0.9967 
Utilization GPU
| ID | GPU | MEM |
------------------
|  0 | 73% | 24% |


Epoch: 9, Acc: 1.00 , Loss: 0.01 100% |//////////////////////| | Time:  0:00:02


Avg Train Accuracy : 1.0000
Test Set Accuracy 0.9967 
Utilization GPU
| ID | GPU | MEM |
------------------
|  0 | 90% | 24% |
Epoch 9 Completed out of 10 loss: 0.011371974616849911
Test Loss: 0.000122


In [12]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['batch',
 'keepprob',
 'input',
 'labels',
 'count',
 'Iterator',
 'IteratorToStringHandle',
 'IteratorGetNext',
 'TensorSliceDataset',
 'BatchDataset',
 'RepeatDataset',
 'make_initializer',
 'TensorSliceDataset_1',
 'BatchDataset_1',
 'make_initializer_1',
 'input_x/shape',
 'input_x',
 'W_conv2/shape',
 'W_conv2/mean',
 'W_conv2/stddev',
 'W_conv2/TruncatedNormal',
 'W_conv2/mul',
 'W_conv2',
 'Variable',
 'Variable/Assign',
 'Variable/read',
 'Rank',
 'range/start',
 'range/delta',
 'range',
 'Min',
 'Rank_1',
 'range_1/start',
 'range_1/delta',
 'range_1',
 'Max',
 'sub',
 'sub_1',
 'truediv',
 'convert_image/Mul/y',
 'convert_image/Mul',
 'convert_image',
 'transpose/perm',
 'transpose',
 'strided_slice/stack',
 'strided_slice/stack_1',
 'strided_slice/stack_2',
 'strided_slice',
 'conv2/filters_0/tag',
 'conv2/filters_0',
 'strided_slice_1/stack',
 'strided_slice_1/stack_1',
 'strided_slice_1/stack_2',
 'strided_slice_1',
 'conv2/filters_1/tag',
 'conv2/filters_1',
 'strided_sl